In [1]:
!date

Wed Sep 19 23:46:47 CST 2018


In [2]:
import os
import os.path as osp
import pandas as pd
import numpy as np

In [3]:
import sys

sys.path.insert(0, "../scripts/")

In [4]:
%load_ext autoreload

In [5]:
%aimport etl

In [6]:
%autoreload 1

In [7]:
from etl import preprocess, extract

In [8]:
pd.set_option("display.max_rows", 20)

In [9]:
concept_mapping = {'Country': 'country',
 'Year': 'year',
 'Cause': 'cause',
 'Sex': 'sex',
 'Deaths1': 'deaths_all_ages',
 'Deaths2': 'deaths_age_0_year',
 'Deaths3': 'deaths_age_1_year',
 'Deaths4': 'deaths_age_2_years',
 'Deaths5': 'deaths_age_3_years',
 'Deaths6': 'deaths_age_4_years',
 'Deaths7': 'deaths_age_5_9_years',
 'Deaths8': 'deaths_age_10_14_years',
 'Deaths9': 'deaths_age_15_19_years',
 'Deaths10': 'deaths_age_20_24_years',
 'Deaths11': 'deaths_age_25_29_years',
 'Deaths12': 'deaths_age_30_34_years',
 'Deaths13': 'deaths_age_35_39_years',
 'Deaths14': 'deaths_age_40_44_years',
 'Deaths15': 'deaths_age_45_49_years',
 'Deaths16': 'deaths_age_50_54_years',
 'Deaths17': 'deaths_age_55_59_years',
 'Deaths18': 'deaths_age_60_64_years',
 'Deaths19': 'deaths_age_65_69_years',
 'Deaths20': 'deaths_age_70_74_years',
 'Deaths21': 'deaths_age_75_79_years',
 'Deaths22': 'deaths_age_80_84_years',
 'Deaths23': 'deaths_age_85_89_years',
 'Deaths24': 'deaths_age_90_94_years',
 'Deaths25': 'deaths_age_95_years_and_above',
 'Deaths26': 'deaths_age_unspecified',
 'IM_deaths1': 'infant_deaths_age_0_day',
 'IM_deaths2': 'infant_deaths_age_1_6_days',
 'IM_deaths3': 'infant_deaths_age_7_27_days',
 'IM_deaths4': 'infant_deaths_age_28_364_days',
 'IM_Deaths1': 'infant_deaths_age_0_day',
 'IM_Deaths2': 'infant_deaths_age_1_6_days',
 'IM_Deaths3': 'infant_deaths_age_7_27_days',
 'IM_Deaths4': 'infant_deaths_age_28_364_days'}

In [10]:
d7 = { "A138": "traffic"
    , "A148": "suicide"
    , "A149": "homicide"
    }

In [11]:
d8 = { "A138": "traffic"
    , "A147": "suicide"
    , "A148": "homicide"
    }

In [12]:
d9 = {"B471": "traffic"
      , "B472": "traffic"  # TODO: late effects from traffic is missing (E929.0). here using B529
      , "B529": "traffic"
     , "B54": "suicide"
     , "B55": "homicide"
     }

In [13]:
d10 = {
    #"Y850": "traffic",
    # "1096": "traffic",
    "1101": "suicide",
    "Y870": "suicide",
    "1102": "homicide",
    "Y871": "homicide"
}

In [14]:
# categories of traffic injures in ICD 10 

traffic = [
    "V011", "V019",
    "V021", "V029",
    "V031", "V039",
    "V041", "V049",
    "V061", "V069",
    "V092", "V093",
    
    "V103", "V104", "V105", "V109",
    "V113", "V114", "V115", "V119",
    "V123", "V124", "V125", "V129",
    "V133", "V134", "V135", "V139",
    "V143", "V144", "V145", "V149",
    "V154", "V155", "V159",
    "V164", "V165", "V169",
    "V174", "V175", "V179",
    "V184", "V185", "V189",
    "V194", "V195", "V196", "V198", "V199",
    
    "V203", "V204", "V205", "V209",
    "V213", "V214", "V215", "V219",
    "V223", "V224", "V225", "V229",
    "V233", "V234", "V235", "V239",
    "V243", "V244", "V245", "V249",
    "V253", "V254", "V255", "V259",
    "V263", "V264", "V265", "V269",
    "V273", "V274", "V275", "V279",
    "V283", "V284", "V285", "V289",
    "V294", "V295", "V296", "V298", "V299",
    
    "V304", "V305", "V306", "V307", "V309",
    "V314", "V315", "V316", "V317", "V319",
    "V324", "V325", "V326", "V327", "V329",
    "V334", "V335", "V336", "V337", "V339",
    "V344", "V345", "V346", "V347", "V349",
    "V354", "V355", "V356", "V357", "V359",
    "V364", "V365", "V366", "V367", "V369",
    "V374", "V375", "V376", "V377", "V379",
    "V384", "V385", "V386", "V387", "V389",
    "V394", "V395", "V396", "V398", "V399",
    
    "V404", "V405", "V406", "V407", "V409",
    "V414", "V415", "V416", "V417", "V419",
    "V424", "V425", "V426", "V427", "V429",
    "V434", "V435", "V436", "V437", "V439",
    "V444", "V445", "V446", "V447", "V449",
    "V454", "V455", "V456", "V457", "V459",
    "V464", "V465", "V466", "V467", "V469",
    "V474", "V475", "V476", "V477", "V479",
    "V484", "V485", "V486", "V487", "V489",
    "V494", "V495", "V496", "V498", "V499",
    
    "V504", "V505", "V506", "V507", "V509",
    "V514", "V515", "V516", "V517", "V519",
    "V524", "V525", "V526", "V527", "V529",
    "V534", "V535", "V536", "V537", "V539",
    "V544", "V545", "V546", "V547", "V549",
    "V554", "V555", "V556", "V557", "V559",
    "V564", "V565", "V566", "V567", "V569",
    "V574", "V575", "V576", "V577", "V579",
    "V584", "V585", "V586", "V587", "V589",
    "V594", "V595", "V596", "V598", "V599",
    
    "V604", "V605", "V606", "V607", "V609",
    "V614", "V615", "V616", "V617", "V619",
    "V624", "V625", "V626", "V627", "V629",
    "V634", "V635", "V636", "V637", "V639",
    "V644", "V645", "V646", "V647", "V649",
    "V654", "V655", "V656", "V657", "V659",
    "V664", "V665", "V666", "V667", "V669",
    "V674", "V675", "V676", "V677", "V679",
    "V684", "V685", "V686", "V687", "V689",
    "V694", "V695", "V696", "V698", "V699",
    
    "V704", "V705", "V706", "V707", "V709",
    "V714", "V715", "V716", "V717", "V719",
    "V724", "V725", "V726", "V727", "V729",
    "V734", "V735", "V736", "V737", "V739",
    "V744", "V745", "V746", "V747", "V749",
    "V754", "V755", "V756", "V757", "V759",
    "V764", "V765", "V766", "V767", "V769",
    "V774", "V775", "V776", "V777", "V779",
    "V784", "V785", "V786", "V787", "V789",
    "V794", "V795", "V796", "V798", "V799",
    
    "V803", "V804", "V805",
    "V811",
    "V821", "V828", "V829",
    "V830", "V831", "V832", "V833",
    "V840", "V841", "V842", "V843",
    "V850", "V851", "V852", "V853",
    "V860", "V861", "V862", "V863",
    "V870", "V871", "V872", "V873", "V874", "V875", "V876", "V877","V878", "V879",

    "V892", "V893",
    "V899",
    "V99",
    "Y850"
]

for i in traffic:
    d10[i] = "traffic"

In [15]:
icd7 = pd.read_csv('../source/morticd07.zip')

/Users/semio/.pyenv/versions/3.6.5/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
df1 = preprocess(icd7, d7.keys())

In [17]:
df1.Cause.unique()

array(['A138', 'A148', 'A149'], dtype=object)

In [18]:
df1

,Country,Year,Cause,Sex,Deaths1,Deaths2,Deaths3,Deaths4,Deaths5,Deaths6,...,Deaths21,Deaths22,Deaths23,Deaths24,Deaths25,Deaths26,IM_Deaths1,IM_Deaths2,IM_Deaths3,IM_Deaths4
266,1125,1955,A138,1,66,1,0,0.0,3.0,2.0,...,0.0,1.0,0.0,NaN,NaN,0,1,NaN,NaN,NaN
267,1125,1955,A138,2,12,0,1,0.0,1.0,0.0,...,0.0,0.0,0.0,NaN,NaN,0,0,NaN,NaN,NaN
286,1125,1955,A148,1,12,0,0,0.0,0.0,0.0,...,0.0,1.0,1.0,NaN,NaN,0,0,NaN,NaN,NaN
287,1125,1955,A148,2,8,0,0,0.0,0.0,0.0,...,0.0,0.0,2.0,NaN,NaN,0,0,NaN,NaN,NaN
288,1125,1955,A149,1,244,1,0,1.0,0.0,0.0,...,2.0,2.0,0.0,NaN,NaN,0,1,NaN,NaN,NaN
289,1125,1955,A149,2,31,1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,0,1,NaN,NaN,NaN
558,1125,1956,A138,1,11,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,0,0,NaN,NaN,NaN
559,1125,1956,A138,2,4,0,1,0.0,0.0,1.0,...,0.0,0.0,0.0,NaN,NaN,0,0,NaN,NaN,NaN
578,1125,1956,A148,1,12,0,0,0.0,0.0,0.0,...,0.0,1.0,0.0,NaN,NaN,0,0,NaN,NaN,NaN
579,1125,1956,A148,2,5,0,0,0.0,0.0,0.0,...,1.0,0.0,2.0,NaN,NaN,0,0,NaN,NaN,NaN


In [19]:
df1.duplicated(subset=["Country", "Year", "Cause", "Sex"]).any()   # should be false

False

In [20]:
df1 = extract(df1, concept_mapping, d7)

In [21]:
df1

deaths_all_ages  deaths_age_0_year  \
country year cause    sex                                       
1125    1955 homicide 1                244                  1   
                      2                 31                  1   
             suicide  1                 12                  0   
                      2                  8                  0   
             traffic  1                 66                  1   
                      2                 12                  0   
        1956 homicide 1                288                  0   
                      2                 34                  0   
             suicide  1                 12                  0   
                      2                  5                  0   
...                                    ...                ...   
5150    1966 suicide  1                156                  0   
                      2                 90                  0   
             traffic  1                427                  0   
                      2                140                  1   
        1967 homicide 1                 21                  3   
                      2                 16                  0   
             suicide  1                191                  0   
                      2                 83                  0   
             traffic  1                453                  4   
                      2                167                  5   

                           deaths_age_1_year  deaths_age_2_years  \
country year cause    sex                                          
1125    1955 homicide 1                    0                 1.0   
                      2                    0                 0.0   
             suicide  1                    0                 0.0   
                      2                    0                 0.0   
             traffic  1                    0                 0.0   
                      2                    1                 0.0   
        1956 homicide 1                    0                 0.0   
                      2                    0                 0.0   
             suicide  1                    0                 0.0   
                      2                    0                 0.0   
...                                      ...                 ...   
5150    1966 suicide  1                    0                 0.0   
                      2                    0                 0.0   
             traffic  1                    3                 4.0   
                      2                    3                 3.0   
        1967 homicide 1                    0                 0.0   
                      2                    2                 0.0   
             suicide  1                    0                 0.0   
                      2                    0                 0.0   
             traffic  1                    4                 0.0   
                      2                    4                 3.0   

                           deaths_age_3_years  deaths_age_4_years  \
country year cause    sex                                           
1125    1955 homicide 1                   0.0                 0.0   
                      2                   0.0                 0.0   
             suicide  1                   0.0                 0.0   
                      2                   0.0                 0.0   
             traffic  1                   3.0                 2.0   
                      2                   1.0                 0.0   
        1956 homicide 1                   2.0                 1.0   
                      2                   1.0                 1.0   
             suicide  1                   0.0                 0.0   
                      2                   0.0                 0.0   
...                                       ...                 ...   
5150    1966 suicide  1                   0.0                 0.0   


In [22]:
icd8 = pd.read_csv("../source/morticd08.zip")

/Users/semio/.pyenv/versions/3.6.5/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
df2 = preprocess(icd8, d8.keys())

In [24]:
df2.duplicated(subset=["Country", "Year", "Cause", "Sex"]).any()   # should be false

False

In [25]:
df2 = extract(df2, concept_mapping, d8)

In [26]:
df2

deaths_all_ages  deaths_age_0_year  \
country year cause    sex                                       
1060    1980 homicide 1                  2                0.0   
                      2                  1                1.0   
             suicide  1                  6                0.0   
                      2                  1                0.0   
             traffic  1                  3                0.0   
                      2                  5                0.0   
1125    1970 traffic  1                432                5.0   
                      2                130                2.0   
        1971 traffic  1                426                2.0   
                      2                120                2.0   
...                                    ...                ...   
5150    1978 suicide  1                208                0.0   
                      2                114                0.0   
             traffic  1                481                2.0   
                      2                207                2.0   
5195    1977 homicide 1                  4                0.0   
                      2                  0                0.0   
             suicide  1                  2                0.0   
                      2                  1                0.0   
             traffic  1                  9                1.0   
                      2                  5                0.0   

                           deaths_age_1_year  deaths_age_2_years  \
country year cause    sex                                          
1060    1980 homicide 1                  0.0                 0.0   
                      2                  0.0                 0.0   
             suicide  1                  0.0                 0.0   
                      2                  0.0                 0.0   
             traffic  1                  0.0                 0.0   
                      2                  1.0                 0.0   
1125    1970 traffic  1                  4.0                 3.0   
                      2                  3.0                 3.0   
        1971 traffic  1                  0.0                 7.0   
                      2                  0.0                 4.0   
...                                      ...                 ...   
5150    1978 suicide  1                  0.0                 0.0   
                      2                  0.0                 0.0   
             traffic  1                  5.0                 3.0   
                      2                  3.0                 3.0   
5195    1977 homicide 1                  0.0                 1.0   
                      2                  0.0                 0.0   
             suicide  1                  0.0                 0.0   
                      2                  0.0                 0.0   
             traffic  1                  0.0                 0.0   
                      2                  0.0                 0.0   

                           deaths_age_3_years  deaths_age_4_years  \
country year cause    sex                                           
1060    1980 homicide 1                   0.0                 0.0   
                      2                   0.0                 0.0   
             suicide  1                   0.0                 0.0   
                      2                   0.0                 0.0   
             traffic  1                   0.0                 0.0   
                      2                   0.0                 0.0   
1125    1970 traffic  1                   5.0                 8.0   
                      2                   3.0                 9.0   
        1971 traffic  1                   6.0                13.0   
                      2                   3.0                 9.0   
...                                       ...                 ...   
5150    1978 suicide  1                   0.0                 0.0   


In [27]:
icd9 = pd.read_csv('../source/morticd9.zip')

/Users/semio/.pyenv/versions/3.6.5/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
df3 = preprocess(icd9, d9.keys())

In [29]:
df3.duplicated(subset=["Country", "Year", "Cause", "Sex"]).any()   # should be False

False

In [30]:
df3.Cause.unique()

array(['B471', 'B54', 'B55', 'B472', 'B529'], dtype=object)

In [31]:
df3 = extract(df3, concept_mapping, d9)

In [32]:
df3

deaths_all_ages  deaths_age_0_year  \
country year cause    sex                                       
1125    1980 homicide 1                294                3.0   
                      2                 77                2.0   
             suicide  1                 10                0.0   
                      2                  5                0.0   
             traffic  1               3496               79.0   
                      2                906               14.0   
        1987 homicide 1                191                0.0   
                      2                 50                0.0   
             suicide  1                 14                0.0   
                      2                  8                0.0   
...                                    ...                ...   
5150    1999 suicide  1                385                0.0   
                      2                131                0.0   
             traffic  1                399                4.0   
                      2                218                7.0   
5195    1980 homicide 1                  3                0.0   
                      2                  1                0.0   
             suicide  1                  2                0.0   
                      2                  3                0.0   
             traffic  1                 29                0.0   
                      2                  9                0.0   

                           deaths_age_1_year  deaths_age_2_years  \
country year cause    sex                                          
1125    1980 homicide 1                  1.0                 1.0   
                      2                  0.0                 0.0   
             suicide  1                  0.0                 0.0   
                      2                  0.0                 0.0   
             traffic  1                  6.0                16.0   
                      2                  4.0                23.0   
        1987 homicide 1                  1.0                 0.0   
                      2                  0.0                 0.0   
             suicide  1                  0.0                 0.0   
                      2                  0.0                 0.0   
...                                      ...                 ...   
5150    1999 suicide  1                  0.0                 0.0   
                      2                  0.0                 0.0   
             traffic  1                  4.0                 2.0   
                      2                  0.0                 1.0   
5195    1980 homicide 1                  0.0                 0.0   
                      2                  0.0                 0.0   
             suicide  1                  0.0                 0.0   
                      2                  0.0                 0.0   
             traffic  1                  0.0                 0.0   
                      2                  0.0                 0.0   

                           deaths_age_3_years  deaths_age_4_years  \
country year cause    sex                                           
1125    1980 homicide 1                   0.0                 4.0   
                      2                   0.0                 0.0   
             suicide  1                   0.0                 0.0   
                      2                   0.0                 0.0   
             traffic  1                  51.0                48.0   
                      2                  29.0                30.0   
        1987 homicide 1                   0.0                 2.0   
                      2                   0.0                 2.0   
             suicide  1                   0.0                 0.0   
                      2                   0.0                 0.0   
...                                       ...                 ...   
5150    1999 suicide  1                   0.0                 0.0   


In [33]:
icd101 = pd.read_csv('../source/Morticd10_part1.zip')

/Users/semio/.pyenv/versions/3.6.5/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
df4 = preprocess(icd101, d10.keys())

In [35]:
df4.duplicated(subset=["Country", "Year", "Cause", "Sex"]).any()   # should be False

False

In [36]:
df4 = extract(df4, concept_mapping, d10)

In [37]:
df4

deaths_all_ages  deaths_age_0_year  \
country year cause    sex                                       
1125    2000 traffic  1                172                0.0   
                      2                 32                0.0   
        2001 traffic  1                 78                0.0   
                      2                 19                0.0   
        2002 traffic  1                 29                0.0   
                      2                  8                0.0   
        2003 traffic  1                  7                0.0   
                      2                  4                0.0   
        2004 traffic  1                 17                1.0   
                      2                  4                0.0   
...                                    ...                ...   
5150    2002 homicide 1                  1                0.0   
             traffic  1                297                1.0   
                      2                142                1.0   
        2003 suicide  1                  2                0.0   
             traffic  1                325                0.0   
                      2                155                1.0   
        2004 homicide 1                  2                0.0   
             suicide  2                  2                0.0   
             traffic  1                305                3.0   
                      2                142                2.0   

                           deaths_age_1_year  deaths_age_2_years  \
country year cause    sex                                          
1125    2000 traffic  1                  5.0                 0.0   
                      2                  9.0                 0.0   
        2001 traffic  1                  7.0                 0.0   
                      2                  4.0                 0.0   
        2002 traffic  1                  2.0                 0.0   
                      2                  1.0                 0.0   
        2003 traffic  1                  0.0                 0.0   
                      2                  2.0                 0.0   
        2004 traffic  1                  2.0                 0.0   
                      2                  1.0                 0.0   
...                                      ...                 ...   
5150    2002 homicide 1                  0.0                 0.0   
             traffic  1                  0.0                 1.0   
                      2                  1.0                 1.0   
        2003 suicide  1                  0.0                 0.0   
             traffic  1                  0.0                 1.0   
                      2                  1.0                 1.0   
        2004 homicide 1                  0.0                 0.0   
             suicide  2                  0.0                 0.0   
             traffic  1                  1.0                 0.0   
                      2                  1.0                 0.0   

                           deaths_age_3_years  deaths_age_4_years  \
country year cause    sex                                           
1125    2000 traffic  1                   0.0                 0.0   
                      2                   0.0                 0.0   
        2001 traffic  1                   0.0                 0.0   
                      2                   0.0                 0.0   
        2002 traffic  1                   0.0                 0.0   
                      2                   0.0                 0.0   
        2003 traffic  1                   0.0                 0.0   
                      2                   0.0                 0.0   
        2004 traffic  1                   0.0                 0.0   
                      2                   0.0                 0.0   
...                                       ...                 ...   
5150    2002 homicide 1                   0.0                 0.0   


In [38]:
icd102 = pd.read_csv('../source/Morticd10_part2.zip')

/Users/semio/.pyenv/versions/3.6.5/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [39]:
df5 = preprocess(icd102, d10.keys())

In [40]:
df5.duplicated(subset=["Country", "Year", "Cause", "Sex"]).any()   # should be False

False

In [41]:
df5 = extract(df5, concept_mapping, d10)

In [42]:
df5

deaths_all_ages  deaths_age_0_year  \
country year cause    sex                                       
1125    2005 traffic  1                 16                0.0   
        2006 traffic  1                 39                0.0   
                      2                 10                0.0   
        2007 traffic  1                110                2.0   
                      2                 41                0.0   
        2008 traffic  1                 43                1.0   
                      2                 17                0.0   
        2009 traffic  1                 18                0.0   
                      2                  2                0.0   
        2010 traffic  1                 26                1.0   
...                                    ...                ...   
5150    2010 traffic  2                114                1.0   
        2011 homicide 1                  1                0.0   
             traffic  1                229                0.0   
                      2                 90                0.0   
        2012 homicide 1                  1                0.0   
             traffic  1                241                0.0   
                      2                 93                0.0   
        2013 homicide 1                  1                0.0   
             traffic  1                192                2.0   
                      2                 81                0.0   

                           deaths_age_1_year  deaths_age_2_years  \
country year cause    sex                                          
1125    2005 traffic  1                  2.0                 0.0   
        2006 traffic  1                  3.0                 0.0   
                      2                  2.0                 0.0   
        2007 traffic  1                  6.0                 0.0   
                      2                  5.0                 0.0   
        2008 traffic  1                  2.0                 0.0   
                      2                  1.0                 0.0   
        2009 traffic  1                  2.0                 0.0   
                      2                  0.0                 0.0   
        2010 traffic  1                  0.0                 0.0   
...                                      ...                 ...   
5150    2010 traffic  2                  0.0                 1.0   
        2011 homicide 1                  0.0                 0.0   
             traffic  1                  0.0                 1.0   
                      2                  0.0                 0.0   
        2012 homicide 1                  0.0                 0.0   
             traffic  1                  1.0                 0.0   
                      2                  0.0                 1.0   
        2013 homicide 1                  0.0                 0.0   
             traffic  1                  0.0                 2.0   
                      2                  0.0                 0.0   

                           deaths_age_3_years  deaths_age_4_years  \
country year cause    sex                                           
1125    2005 traffic  1                   0.0                 0.0   
        2006 traffic  1                   0.0                 0.0   
                      2                   0.0                 0.0   
        2007 traffic  1                   0.0                 0.0   
                      2                   0.0                 0.0   
        2008 traffic  1                   0.0                 0.0   
                      2                   0.0                 0.0   
        2009 traffic  1                   0.0                 0.0   
                      2                   0.0                 0.0   
        2010 traffic  1                   0.0                 0.0   
...                                       ...                 ...   
5150    2010 traffic  2                   0.0                 0.0   


In [43]:
df_all = pd.concat([df1, df2, df3, df4, df5])

In [44]:
df_all

deaths_all_ages  deaths_age_0_year  \
country year cause    sex                                       
1125    1955 homicide 1                244                1.0   
                      2                 31                1.0   
             suicide  1                 12                0.0   
                      2                  8                0.0   
             traffic  1                 66                1.0   
                      2                 12                0.0   
        1956 homicide 1                288                0.0   
                      2                 34                0.0   
             suicide  1                 12                0.0   
                      2                  5                0.0   
...                                    ...                ...   
5150    2010 traffic  2                114                1.0   
        2011 homicide 1                  1                0.0   
             traffic  1                229                0.0   
                      2                 90                0.0   
        2012 homicide 1                  1                0.0   
             traffic  1                241                0.0   
                      2                 93                0.0   
        2013 homicide 1                  1                0.0   
             traffic  1                192                2.0   
                      2                 81                0.0   

                           deaths_age_1_year  deaths_age_2_years  \
country year cause    sex                                          
1125    1955 homicide 1                  0.0                 1.0   
                      2                  0.0                 0.0   
             suicide  1                  0.0                 0.0   
                      2                  0.0                 0.0   
             traffic  1                  0.0                 0.0   
                      2                  1.0                 0.0   
        1956 homicide 1                  0.0                 0.0   
                      2                  0.0                 0.0   
             suicide  1                  0.0                 0.0   
                      2                  0.0                 0.0   
...                                      ...                 ...   
5150    2010 traffic  2                  0.0                 1.0   
        2011 homicide 1                  0.0                 0.0   
             traffic  1                  0.0                 1.0   
                      2                  0.0                 0.0   
        2012 homicide 1                  0.0                 0.0   
             traffic  1                  1.0                 0.0   
                      2                  0.0                 1.0   
        2013 homicide 1                  0.0                 0.0   
             traffic  1                  0.0                 2.0   
                      2                  0.0                 0.0   

                           deaths_age_3_years  deaths_age_4_years  \
country year cause    sex                                           
1125    1955 homicide 1                   0.0                 0.0   
                      2                   0.0                 0.0   
             suicide  1                   0.0                 0.0   
                      2                   0.0                 0.0   
             traffic  1                   3.0                 2.0   
                      2                   1.0                 0.0   
        1956 homicide 1                   2.0                 1.0   
                      2                   1.0                 1.0   
             suicide  1                   0.0                 0.0   
                      2                   0.0                 0.0   
...                                       ...                 ...   
5150    2010 traffic  2                   0.0                 0.0   


In [66]:
# population

from http://apps.who.int/healthinfo/statistics/mortality/whodpms/definitions/pop.htm

In [45]:
pop_div = pd.read_csv("../source/pop.csv")

In [46]:
pop_div

,All ages,"100,000"
0,0,1822.0
1,01-04,7033.0
2,05-09,8687.0
3,10-14,8597.0
4,15-19,8474.0
5,20-24,8222.0
6,25-29,7928.0
7,30-34,7605.0
8,35-39,7145.0
9,40-44,6590.0


In [47]:
pop_concepts = pd.read_csv('../source/concepts_pop.csv')

In [48]:
pop_concepts = pop_concepts.dropna(how='all', axis=1).copy()

In [49]:
pop_concepts

,Column name,Content
0,Country,Country”
1,Year,Year to which data refer
2,Sex,"1 male, 2 female"
3,Pop1,Population at all ages
4,Pop2,Population at age 0 year
5,Pop3,Population at age1 year
6,Pop4,Population at age 2 years
7,Pop5,Population at age 3 years
8,Pop6,Population at age 4 years
9,Pop7,Population at age 5-9 years


In [50]:
from ddf_utils.str import to_concept_id

In [51]:
pop_concepts['concept'] = pop_concepts['Column name'].map(to_concept_id)

In [52]:
pop_concepts.loc[3:, 'concept'] = pop_concepts.loc[3:, 'Content'].map(lambda x: to_concept_id(x.replace("at ", "")))

In [53]:
pop_concepts

,Column name,Content,concept
0,Country,Country”,country
1,Year,Year to which data refer,year
2,Sex,"1 male, 2 female",sex
3,Pop1,Population at all ages,population_all_ages
4,Pop2,Population at age 0 year,population_age_0_year
5,Pop3,Population at age1 year,population_age1_year
6,Pop4,Population at age 2 years,population_age_2_years
7,Pop5,Population at age 3 years,population_age_3_years
8,Pop6,Population at age 4 years,population_age_4_years
9,Pop7,Population at age 5-9 years,population_age_5_9_years


In [54]:
pop_concept_mapping = pop_concepts.set_index('Column name')['concept'].to_dict()

In [55]:
pop_concept_mapping

{'Country': 'country',
 'Year': 'year',
 'Sex': 'sex',
 'Pop1': 'population_all_ages',
 'Pop2': 'population_age_0_year',
 'Pop3': 'population_age1_year',
 'Pop4': 'population_age_2_years',
 'Pop5': 'population_age_3_years',
 'Pop6': 'population_age_4_years',
 'Pop7': 'population_age_5_9_years',
 'Pop8': 'population_age_10_14_years',
 'Pop9': 'population_age_15_19_years',
 'Pop10': 'population_age_20_24_years',
 'Pop11': 'population_age_25_29_years',
 'Pop12': 'population_age_30_34_years',
 'Pop13': 'population_age_35_39_years',
 'Pop14': 'population_age_40_44_years',
 'Pop15': 'population_age_45_49_years',
 'Pop16': 'population_age_50_54_years',
 'Pop17': 'population_age_55_59_years',
 'Pop18': 'population_age_60_64_years',
 'Pop19': 'population_age_65_69_years',
 'Pop20': 'population_age_70_74_years',
 'Pop21': 'population_age_75_79_years',
 'Pop22': 'population_age_80_84_years',
 'Pop23': 'population_age_85_89_years',
 'Pop24': 'population_age_90_94_years',
 'Pop25': 'population_age_

In [56]:
pop = pd.read_csv('../source/Pop.zip')

In [57]:
from etl import is_country

In [58]:
pop

,Country,Admin1,SubDiv,Year,Sex,Frmat,Pop1,Pop2,Pop3,Pop4,...,Pop18,Pop19,Pop20,Pop21,Pop22,Pop23,Pop24,Pop25,Pop26,Lb
0,1060,NaN,NaN,1980,1,7,137100.0,3400.0,15800.0,NaN,...,NaN,5300.0,NaN,2900.0,NaN,NaN,NaN,NaN,6500.0,5000.0
1,1060,NaN,NaN,1980,2,7,159000.0,4000.0,18400.0,NaN,...,NaN,6200.0,NaN,3400.0,NaN,NaN,NaN,NaN,7500.0,6000.0
2,1125,NaN,NaN,1955,1,2,5051500.0,150300.0,543400.0,NaN,...,110200.0,51100.0,41600.0,14300.0,11800.0,25300.0,NaN,NaN,0.0,253329.0
3,1125,NaN,NaN,1955,2,2,5049400.0,145200.0,551000.0,NaN,...,122100.0,51100.0,50700.0,15800.0,18000.0,28500.0,NaN,NaN,0.0,237901.0
4,1125,NaN,NaN,1956,1,2,5353700.0,158700.0,576600.0,NaN,...,116900.0,54100.0,44000.0,14900.0,12400.0,26600.0,NaN,NaN,0.0,250022.0
5,1125,NaN,NaN,1956,2,2,5351400.0,153600.0,584800.0,NaN,...,129500.0,54300.0,53800.0,16700.0,19000.0,29400.0,NaN,NaN,0.0,235887.0
6,1125,NaN,NaN,1957,1,2,5403000.0,160300.0,580800.0,NaN,...,118400.0,54100.0,44100.0,14900.0,12400.0,26500.0,NaN,NaN,0.0,231412.0
7,1125,NaN,NaN,1957,2,2,5392000.0,155300.0,589400.0,NaN,...,130000.0,54000.0,53500.0,16400.0,18700.0,29800.0,NaN,NaN,0.0,215811.0
8,1125,NaN,NaN,1958,1,2,5506900.0,162800.0,592000.0,NaN,...,120600.0,54600.0,44500.0,15100.0,12500.0,26700.0,NaN,NaN,0.0,255600.0
9,1125,NaN,NaN,1958,2,2,5494400.0,157800.0,600100.0,NaN,...,132400.0,54700.0,54100.0,16600.0,18800.0,30200.0,NaN,NaN,0.0,238109.0


In [59]:
pop.columns

Index(['Country', 'Admin1', 'SubDiv', 'Year', 'Sex', 'Frmat', 'Pop1', 'Pop2',
       'Pop3', 'Pop4', 'Pop5', 'Pop6', 'Pop7', 'Pop8', 'Pop9', 'Pop10',
       'Pop11', 'Pop12', 'Pop13', 'Pop14', 'Pop15', 'Pop16', 'Pop17', 'Pop18',
       'Pop19', 'Pop20', 'Pop21', 'Pop22', 'Pop23', 'Pop24', 'Pop25', 'Pop26',
       'Lb'],
      dtype='object')

In [60]:
pop["is_country"] = is_country(pop)

In [61]:
pop = pop[pop.is_country == True]

In [62]:
cols = ['Country', 'Year', 'Sex', 'Pop1', 'Pop2',
       'Pop3', 'Pop4', 'Pop5', 'Pop6', 'Pop7', 'Pop8', 'Pop9', 'Pop10',
       'Pop11', 'Pop12', 'Pop13', 'Pop14', 'Pop15', 'Pop16', 'Pop17', 'Pop18',
       'Pop19', 'Pop20', 'Pop21', 'Pop22', 'Pop23', 'Pop24', 'Pop25', 'Pop26',
       'Lb']

In [63]:
pop = pop[cols].copy()

In [64]:
pop.columns = pop.columns.map(pop_concept_mapping)

In [65]:
pop.columns

Index(['country', 'year', 'sex', 'population_all_ages',
       'population_age_0_year', 'population_age1_year',
       'population_age_2_years', 'population_age_3_years',
       'population_age_4_years', 'population_age_5_9_years',
       'population_age_10_14_years', 'population_age_15_19_years',
       'population_age_20_24_years', 'population_age_25_29_years',
       'population_age_30_34_years', 'population_age_35_39_years',
       'population_age_40_44_years', 'population_age_45_49_years',
       'population_age_50_54_years', 'population_age_55_59_years',
       'population_age_60_64_years', 'population_age_65_69_years',
       'population_age_70_74_years', 'population_age_75_79_years',
       'population_age_80_84_years', 'population_age_85_89_years',
       'population_age_90_94_years', 'population_age_95_years_and_over',
       'population_age_unspecified', 'live_births'],
      dtype='object')

In [66]:
pop

,country,year,sex,population_all_ages,population_age_0_year,population_age1_year,population_age_2_years,population_age_3_years,population_age_4_years,population_age_5_9_years,...,population_age_60_64_years,population_age_65_69_years,population_age_70_74_years,population_age_75_79_years,population_age_80_84_years,population_age_85_89_years,population_age_90_94_years,population_age_95_years_and_over,population_age_unspecified,live_births
0,1060,1980,1,137100.0,3400.0,15800.0,NaN,NaN,NaN,39200.0,...,NaN,5300.0,NaN,2900.0,NaN,NaN,NaN,NaN,6500.0,5000.0
1,1060,1980,2,159000.0,4000.0,18400.0,NaN,NaN,NaN,45400.0,...,NaN,6200.0,NaN,3400.0,NaN,NaN,NaN,NaN,7500.0,6000.0
2,1125,1955,1,5051500.0,150300.0,543400.0,NaN,NaN,NaN,615300.0,...,110200.0,51100.0,41600.0,14300.0,11800.0,25300.0,NaN,NaN,0.0,253329.0
3,1125,1955,2,5049400.0,145200.0,551000.0,NaN,NaN,NaN,613400.0,...,122100.0,51100.0,50700.0,15800.0,18000.0,28500.0,NaN,NaN,0.0,237901.0
4,1125,1956,1,5353700.0,158700.0,576600.0,NaN,NaN,NaN,653000.0,...,116900.0,54100.0,44000.0,14900.0,12400.0,26600.0,NaN,NaN,0.0,250022.0
5,1125,1956,2,5351400.0,153600.0,584800.0,NaN,NaN,NaN,650900.0,...,129500.0,54300.0,53800.0,16700.0,19000.0,29400.0,NaN,NaN,0.0,235887.0
6,1125,1957,1,5403000.0,160300.0,580800.0,NaN,NaN,NaN,657600.0,...,118400.0,54100.0,44100.0,14900.0,12400.0,26500.0,NaN,NaN,0.0,231412.0
7,1125,1957,2,5392000.0,155300.0,589400.0,NaN,NaN,NaN,655400.0,...,130000.0,54000.0,53500.0,16400.0,18700.0,29800.0,NaN,NaN,0.0,215811.0
8,1125,1958,1,5506900.0,162800.0,592000.0,NaN,NaN,NaN,670000.0,...,120600.0,54600.0,44500.0,15100.0,12500.0,26700.0,NaN,NaN,0.0,255600.0
9,1125,1958,2,5494400.0,157800.0,600100.0,NaN,NaN,NaN,667400.0,...,132400.0,54700.0,54100.0,16600.0,18800.0,30200.0,NaN,NaN,0.0,238109.0


In [74]:
pop_ = pop.copy()

In [68]:
pop_div

,All ages,"100,000"
0,0,1822.0
1,01-04,7033.0
2,05-09,8687.0
3,10-14,8597.0
4,15-19,8474.0
5,20-24,8222.0
6,25-29,7928.0
7,30-34,7605.0
8,35-39,7145.0
9,40-44,6590.0


In [69]:
pop_div_t = pop_div.set_index('All ages').T.reset_index(drop=True)

In [70]:
pop_div_t

All ages,0,01-04,05-09,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+
0,1822.0,7033.0,8687.0,8597.0,8474.0,8222.0,7928.0,7605.0,7145.0,6590.0,6038.0,5371.0,4547.0,3723.0,2955.0,2210.0,1515.0,905.0,632.0


In [71]:
pop_div_t.sum(axis=1)

0    99999.0
dtype: float64

In [86]:
pop_ = pop_.rename(columns={'population_age1_year': 'population_age_1_year'})
pop_ = pop_.set_index(['country', 'year', 'sex'])

In [ ]:
# 1014

In [87]:
df = df_all.query("cause == 'traffic'").copy()
df.index = df.index.droplevel(2)

In [88]:
pop0 = pop_['population_age_0_year']
deaths0 = df['deaths_age_0_year']

pop0104 = pop_[['population_age_1_year',
       'population_age_2_years', 'population_age_3_years',
       'population_age_4_years',]].sum(axis=1)
deaths0104 = df[['deaths_age_1_year',
       'deaths_age_2_years', 'deaths_age_3_years', 'deaths_age_4_years']].sum(axis=1)

pop0509 = pop_['population_age_5_9_years']
deaths0509 = df['deaths_age_5_9_years']

pop1014 = pop_['population_age_10_14_years']
deaths1014 = df['deaths_age_10_14_years']

In [89]:
pop014 = pop_[['population_age_0_year', 'population_age_1_year',
       'population_age_2_years', 'population_age_3_years',
       'population_age_4_years', 'population_age_5_9_years',
       'population_age_10_14_years']]

In [90]:
stdPop0 = pop_div_t.loc[0, '0']
stdPop0104 = pop_div_t.loc[0, '01-04']
stdPop0509 = pop_div_t.loc[0, '05-09']
stdPop1014 = pop_div_t.loc[0, '10-14']

stdPop014 = pop_div_t.loc[0][:4].sum()

In [91]:
deaths0 / pop0 * (stdPop0 / stdPop014) + \
deaths0104 / pop0104 * (stdPop0104 / stdPop014) + \
deaths0509 / pop0509 * (stdPop0509 / stdPop014) + \
deaths1014 / pop1014 * (stdPop1014 / stdPop014)

country  year  sex
1060     1980  1               NaN
               2               NaN
1125     1955  1      2.145764e-05
               2      3.685618e-06
         1956  1      3.080849e-06
               2      1.430766e-06
         1957  1      2.042195e-06
               2      5.070776e-07
         1958  1      1.426896e-05
               2      7.660285e-06
                          ...     
5150     2012  1      2.432437e-05
               2      6.869403e-06
         2013  1      1.724037e-05
               2      0.000000e+00
5195     1977  1      3.075302e-06
               2      0.000000e+00
         1980  1      0.000000e+00
               2      0.000000e+00
5200     2003  1               NaN
               2               NaN
Length: 11494, dtype: float64

In [92]:
rate014 = (deaths0 / pop0 * (stdPop0 / stdPop014) + \
deaths0104 / pop0104 * (stdPop0104 / stdPop014) + \
deaths0509 / pop0509 * (stdPop0509 / stdPop014) + \
deaths1014 / pop1014 * (stdPop1014 / stdPop014)) * 100000

In [98]:
rate014.dropna().reset_index().query("country == '1125'").year.unique()

array([1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965,
       1966, 1967, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978,
       1979, 1980, 1987, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       2009, 2010, 2011, 2012, 2013, 2014, 2015])

In [ ]:
#1529

In [105]:
pop1519 = pop_['population_age_15_19_years']
pop2024 = pop_['population_age_20_24_years']
pop2529 = pop_['population_age_25_29_years']

deaths1519 = df['deaths_age_15_19_years']
deaths2024 = df['deaths_age_20_24_years']
deaths2529 = df['deaths_age_25_29_years']

stdPop1519 = pop_div_t.loc[0, '15-19']
stdPop2024 = pop_div_t.loc[0, '20-24']
stdPop2529 = pop_div_t.loc[0, '25-29']

stdPop1529 = pop_div_t.loc[0][4:7].sum()

In [106]:
def rate(death, pop, stdPop, stdPopSum):
    return death / pop * (stdPop / stdPopSum)

In [107]:
rate1529 = (rate(deaths1519, pop1519, stdPop1519, stdPop1529) + \
rate(deaths2024, pop2024, stdPop2024, stdPop1529) + \
rate(deaths2529, pop2529, stdPop2529, stdPop1529) ) * 100000

In [112]:
rate1529.dropna().reset_index().query("country == '1125' and sex == 1").year.unique()

array([1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965,
       1966, 1967, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978,
       1979, 1980, 1987, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       2009, 2010, 2011, 2012, 2013, 2014, 2015])

In [114]:
params = []
for i in ['15-19', '20-24', '25-29']:
    pop_g = pop_['population_age_{}_years'.format(i.replace('-', '_'))]
    deaths_g = df['deaths_age_{}_years'.format(i.replace('-', '_'))]
    std_pop_g = pop_div_t.loc[0, i]
    params.append([pop_g, deaths_g, std_pop_g])

In [115]:
total_std_pop = sum(x[-1] for x in params)

In [116]:
rate1529_2 = []

for ps in params:
    rate1529_2.append(rate(ps[1], ps[0], ps[2], total_std_pop))

In [117]:
rate1529_2 = sum(rate1529_2) * 100000

In [118]:
rate1529_2.dropna().reset_index().query("country == '1125' and sex == 1").year.unique()

array([1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965,
       1966, 1967, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978,
       1979, 1980, 1987, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       2009, 2010, 2011, 2012, 2013, 2014, 2015])

In [317]:
# 3044

In [99]:
params = []
for i in ['30-34', '35-39', '40-44']:
    pop_g = pop_['population_age_{}_years'.format(i.replace('-', '_'))]
    deaths_g = df['deaths_age_{}_years'.format(i.replace('-', '_'))]
    std_pop_g = pop_div_t.loc[0, i]
    params.append([pop_g, deaths_g, std_pop_g])

In [100]:
total_std_pop = sum(x[-1] for x in params)

In [101]:
total_std_pop

21340.0

In [102]:
rate3044 = []

for ps in params:
    rate3044.append(rate(ps[1], ps[0], ps[2], total_std_pop))

NameError: name 'rate' is not defined

In [103]:
rate3044 = sum(rate3044) * 100000

In [351]:
rate3044.reset_index().query("country == '2450' and year == 2005")

,country,year,sex,0
4372,2450,2005,1,22.707078
4373,2450,2005,2,8.591689


In [352]:
# 45_59

In [353]:
params = []
for i in ['45-49', '50-54', '55-59']:
    pop_g = pop_['population_age_{}_years'.format(i.replace('-', '_'))]
    deaths_g = df['deaths_age_{}_years'.format(i.replace('-', '_'))]
    std_pop_g = pop_div_t.loc[0, i]
    params.append([pop_g, deaths_g, std_pop_g])

In [354]:
total_std_pop = sum(x[-1] for x in params)

In [355]:
rate4559 = []

for ps in params:
    rate4559.append(rate(ps[1], ps[0], ps[2], total_std_pop))

In [356]:
rate4559 = sum(rate4559) * 100000

In [357]:
rate4559.reset_index().query("country == '4010' and year == 2005")

,country,year,sex,0
6191,4010,2005,1,11.853158
6192,4010,2005,2,3.080719


In [ ]:
# 60+

In [135]:
pop_[[
    'population_age_60_64_years',
        'population_age_85_89_years',
        'population_age_90_94_years', 
        'population_age_95_years_and_over'
    ]].query('country == "1125"').dropna(how='all')

population_age_60_64_years  population_age_85_89_years  \
country year sex                                                           
1125    1955 1                      110200.0                 25300.00000   
             2                      122100.0                 28500.00000   
        1956 1                      116900.0                 26600.00000   
             2                      129500.0                 29400.00000   
        1957 1                      118400.0                 26500.00000   
             2                      130000.0                 29800.00000   
        1958 1                      120600.0                 26700.00000   
             2                      132400.0                 30200.00000   
        1959 1                      122700.0                 27000.00000   
             2                      134800.0                 30600.00000   
...                                      ...                         ...   
        2011 1                      991839.0                         NaN   
             2                      889603.0                         NaN   
        2012 1                     1017278.0                         NaN   
             2                      912511.0                         NaN   
        2013 1                     1167988.0                         NaN   
             2                     1185041.0                         NaN   
        2014 1                     1129991.0                 58233.86187   
             2                     1131325.0                 62046.81316   
        2015 1                     1157477.0                 59650.00000   
             2                     1159752.0                 63604.08643   

                  population_age_90_94_years  population_age_95_years_and_over  
country year sex                                                                
1125    1955 1                           NaN                               NaN  
             2                           NaN                               NaN  
        1956 1                           NaN                               NaN  
             2                           NaN                               NaN  
        1957 1                           NaN                               NaN  
             2                           NaN                               NaN  
        1958 1                           NaN                               NaN  
             2                           NaN                               NaN  
        1959 1                           NaN                               NaN  
             2                           NaN                               NaN  
...                                      ...                               ...  
        2011 1                           NaN                               NaN  
             2                           NaN                               NaN  
        2012 1                           NaN                               NaN  
             2                           NaN                               NaN  
        2013 1                           NaN                               NaN  
             2                           NaN                               NaN  
        2014 1                   20349.00000                        6628.00000  
             2                   29711.00000                       11935.00000  
        2015 1                   20844.00000                        6789.00000  
             2                   30453.64272                       12232.62746  

[96 rows x 4 columns]

In [119]:
gs = [
    {'spec': '60_64', 'std': '60-64'},
    {'spec': '65_69', 'std': '65-69'},
    {'spec': '70_74', 'std': '70-74'},
    {'spec': '75_79', 'std': '75-79'},
    {'spec': '80_84', 'std': '80-84'},
    {'spec': [
        'population_age_85_89_years',
        'population_age_90_94_years', 
        'population_age_95_years_and_over'
    ], 'std': '85+'}
]

params = []
for i in gs:
    if isinstance(i['spec'], str):
        pop_g = pop_['population_age_{}_years'.format(i['spec'].replace('-', '_'))]
        deaths_g = df['deaths_age_{}_years'.format(i['spec'].replace('-', '_'))]
    else:
        pop_g = pop_[i['spec']].sum(axis=1)
        deaths_g = df[[x.replace('population', 'deaths').replace('over', 'above') for x in i['spec']]].sum(axis=1)
    
    std_pop_g = pop_div_t.loc[0, i['std']]
    params.append([pop_g, deaths_g, std_pop_g])

In [120]:
total_std_pop = sum(x[-1] for x in params)

In [136]:
rate60p = []

for ps in params:
    rate60p.append(rate(ps[1], ps[0], ps[2], total_std_pop))

In [137]:
r0 = rate60p.pop()

In [138]:
for r in rate60p:
    r0 = r0.append(r)

In [139]:
r0 = r0 * 100000

In [141]:
r0.dropna().reset_index().query("country == '1125'").year.unique()

array([1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 2004, 2005,
       2006, 2007, 2008, 2014, 2015, 1964, 1965, 1966, 1967, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1987, 2001,
       2002, 2003, 2009, 2010, 2011, 2012, 2013])

In [122]:
rate60p = sum(rate60p) * 100000

In [131]:
rate60p.dropna().reset_index().query("country == '1125'").year.unique()

array([1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 2004, 2005,
       2006, 2007, 2008, 2014, 2015])

In [371]:
df_all

deaths_all_ages  deaths_age_0_year  \
country year cause    sex                                       
1125    1955 homicide 1                244                1.0   
                      2                 31                1.0   
             suicide  1                 12                0.0   
                      2                  8                0.0   
             traffic  1                 66                1.0   
                      2                 12                0.0   
        1956 homicide 1                288                0.0   
                      2                 34                0.0   
             suicide  1                 12                0.0   
                      2                  5                0.0   
...                                    ...                ...   
5150    2010 traffic  2                114                1.0   
        2011 homicide 1                  1                0.0   
             traffic  1                229                0.0   
                      2                 90                0.0   
        2012 homicide 1                  1                0.0   
             traffic  1                241                0.0   
                      2                 93                0.0   
        2013 homicide 1                  1                0.0   
             traffic  1                192                2.0   
                      2                 81                0.0   

                           deaths_age_1_year  deaths_age_2_years  \
country year cause    sex                                          
1125    1955 homicide 1                  0.0                 1.0   
                      2                  0.0                 0.0   
             suicide  1                  0.0                 0.0   
                      2                  0.0                 0.0   
             traffic  1                  0.0                 0.0   
                      2                  1.0                 0.0   
        1956 homicide 1                  0.0                 0.0   
                      2                  0.0                 0.0   
             suicide  1                  0.0                 0.0   
                      2                  0.0                 0.0   
...                                      ...                 ...   
5150    2010 traffic  2                  0.0                 1.0   
        2011 homicide 1                  0.0                 0.0   
             traffic  1                  0.0                 1.0   
                      2                  0.0                 0.0   
        2012 homicide 1                  0.0                 0.0   
             traffic  1                  1.0                 0.0   
                      2                  0.0                 1.0   
        2013 homicide 1                  0.0                 0.0   
             traffic  1                  0.0                 2.0   
                      2                  0.0                 0.0   

                           deaths_age_3_years  deaths_age_4_years  \
country year cause    sex                                           
1125    1955 homicide 1                   0.0                 0.0   
                      2                   0.0                 0.0   
             suicide  1                   0.0                 0.0   
                      2                   0.0                 0.0   
             traffic  1                   3.0                 2.0   
                      2                   1.0                 0.0   
        1956 homicide 1                   2.0                 1.0   
                      2                   1.0                 1.0   
             suicide  1                   0.0                 0.0   
                      2                   0.0                 0.0   
...                                       ...                 ...   
5150    2010 traffic  2                   0.0                 0.0   


In [372]:
# serving

In [142]:
from etl import rate

In [143]:
pop_.columns

Index(['population_all_ages', 'population_age_0_year', 'population_age_1_year',
       'population_age_2_years', 'population_age_3_years',
       'population_age_4_years', 'population_age_5_9_years',
       'population_age_10_14_years', 'population_age_15_19_years',
       'population_age_20_24_years', 'population_age_25_29_years',
       'population_age_30_34_years', 'population_age_35_39_years',
       'population_age_40_44_years', 'population_age_45_49_years',
       'population_age_50_54_years', 'population_age_55_59_years',
       'population_age_60_64_years', 'population_age_65_69_years',
       'population_age_70_74_years', 'population_age_75_79_years',
       'population_age_80_84_years', 'population_age_85_89_years',
       'population_age_90_94_years', 'population_age_95_years_and_over',
       'population_age_unspecified', 'live_births'],
      dtype='object')

In [144]:
gs014 = [
        {'spec': ['population_age_0_year'], 'std': '0'},
        {
            'spec': [
                'population_age_1_year',
                'population_age_2_years', 'population_age_3_years',
                'population_age_4_years'
            ],
            'std': '01-04'},
        {'spec': '5_9', 'std': '05-09'},
        {'spec': '10_14', 'std': '10-14'}
    ]

gs1529 = [
        {'spec': '15_19', 'std': '15-19'},
        {'spec': '20_24', 'std': '20-24'},
        {'spec': '25_29', 'std': '25-29'}
    ]

gs3044 = [
        {'spec': '30_34', 'std': '30-34'},
        {'spec': '35_39', 'std': '35-39'},
        {'spec': '40_44', 'std': '40-44'}
    ]

gs4559 = [
        {'spec': '45_49', 'std': '45-49'},
        {'spec': '50_54', 'std': '50-54'},
        {'spec': '55_59', 'std': '55-59'}
    ]

gs60plus = [
        {'spec': '60_64', 'std': '60-64'},
        {'spec': '65_69', 'std': '65-69'},
        {'spec': '70_74', 'std': '70-74'},
        {'spec': '75_79', 'std': '75-79'},
        {'spec': '80_84', 'std': '80-84'},
        {'spec': [
            'population_age_85_89_years',
            'population_age_90_94_years',
            'population_age_95_years_and_over'
        ], 'std': '85+'}
    ]

In [394]:
# creat both_sex for sex entity

In [145]:
deaths = df_all.copy()

In [146]:
deaths_bothsex = deaths.groupby(level=[0, 1, 2]).agg('sum')

In [147]:
deaths_bothsex['sex'] = 0
deaths_bothsex = deaths_bothsex.set_index('sex', append=True)

In [148]:
deaths = deaths.append(deaths_bothsex)

In [149]:
deaths = deaths.sort_index()

In [150]:
deaths

deaths_all_ages  deaths_age_0_year  \
country year cause    sex                                       
1060    1980 homicide 0                  3                1.0   
                      1                  2                0.0   
                      2                  1                1.0   
             suicide  0                  7                0.0   
                      1                  6                0.0   
                      2                  1                0.0   
             traffic  0                  8                0.0   
                      1                  3                0.0   
                      2                  5                0.0   
1125    1955 homicide 0                275                2.0   
...                                    ...                ...   
5195    1977 traffic  2                  5                0.0   
        1980 homicide 0                  4                0.0   
                      1                  3                0.0   
                      2                  1                0.0   
             suicide  0                  5                0.0   
                      1                  2                0.0   
                      2                  3                0.0   
             traffic  0                 38                0.0   
                      1                 29                0.0   
                      2                  9                0.0   

                           deaths_age_1_year  deaths_age_2_years  \
country year cause    sex                                          
1060    1980 homicide 0                  0.0                 0.0   
                      1                  0.0                 0.0   
                      2                  0.0                 0.0   
             suicide  0                  0.0                 0.0   
                      1                  0.0                 0.0   
                      2                  0.0                 0.0   
             traffic  0                  1.0                 0.0   
                      1                  0.0                 0.0   
                      2                  1.0                 0.0   
1125    1955 homicide 0                  0.0                 1.0   
...                                      ...                 ...   
5195    1977 traffic  2                  0.0                 0.0   
        1980 homicide 0                  0.0                 0.0   
                      1                  0.0                 0.0   
                      2                  0.0                 0.0   
             suicide  0                  0.0                 0.0   
                      1                  0.0                 0.0   
                      2                  0.0                 0.0   
             traffic  0                  0.0                 0.0   
                      1                  0.0                 0.0   
                      2                  0.0                 0.0   

                           deaths_age_3_years  deaths_age_4_years  \
country year cause    sex                                           
1060    1980 homicide 0                   0.0                 0.0   
                      1                   0.0                 0.0   
                      2                   0.0                 0.0   
             suicide  0                   0.0                 0.0   
                      1                   0.0                 0.0   
                      2                   0.0                 0.0   
             traffic  0                   0.0                 0.0   
                      1                   0.0                 0.0   
                      2                   0.0                 0.0   
1125    1955 homicide 0                   0.0                 0.0   
...                                       ...                 ...   
5195    1977 traffic  2                   0.0                 0.0   


In [151]:
pop_bothsex = pop_.groupby(level=[0, 1]).agg('sum')

In [152]:
pop_bothsex['sex'] = 0
pop_bothsex = pop_bothsex.set_index('sex', append=True)

In [153]:
pop_ = pop_.append(pop_bothsex)
pop_ = pop_.sort_index()

In [154]:
pop_

population_all_ages  population_age_0_year  \
country year sex                                               
1060    1980 0               296100.0                 7400.0   
             1               137100.0                 3400.0   
             2               159000.0                 4000.0   
1125    1955 0             10100900.0               295500.0   
             1              5051500.0               150300.0   
             2              5049400.0               145200.0   
        1956 0             10705100.0               312300.0   
             1              5353700.0               158700.0   
             2              5351400.0               153600.0   
        1957 0             10795000.0               315600.0   
...                               ...                    ...   
5150    2013 2              2269800.0                29380.0   
5195    1977 0              2809000.0               108800.0   
             1              1451000.0                55400.0   
             2              1358000.0                53400.0   
        1980 0              3010700.0               102700.0   
             1              1575700.0                53400.0   
             2              1435000.0                49300.0   
5200    2003 0               101404.0                 2221.0   
             1                51711.0                 1156.0   
             2                49693.0                 1065.0   

                  population_age_1_year  population_age_2_years  \
country year sex                                                  
1060    1980 0                  34200.0                     0.0   
             1                  15800.0                     NaN   
             2                  18400.0                     NaN   
1125    1955 0                1094400.0                     0.0   
             1                 543400.0                     NaN   
             2                 551000.0                     NaN   
        1956 0                1161400.0                     0.0   
             1                 576600.0                     NaN   
             2                 584800.0                     NaN   
        1957 0                1170200.0                     0.0   
...                                 ...                     ...   
5150    2013 2                  29960.0                 30590.0   
5195    1977 0                 392100.0                     0.0   
             1                 199900.0                     NaN   
             2                 192200.0                     NaN   
        1980 0                  90600.0                 93600.0   
             1                  47400.0                 48900.0   
             2                  43200.0                 44700.0   
5200    2003 0                   9758.0                     0.0   
             1                   5045.0                     NaN   
             2                   4713.0                     NaN   

                  population_age_3_years  population_age_4_years  \
country year sex                                                   
1060    1980 0                       0.0                     0.0   
             1                       NaN                     NaN   
             2                       NaN                     NaN   
1125    1955 0                       0.0                     0.0   
             1                       NaN                     NaN   
             2                       NaN                     NaN   
        1956 0                       0.0                     0.0   
             1                       NaN                     NaN   
             2                       NaN                     NaN   
        1957 0                       0.0                     0.0   
...                                  ...                     ...   
5150    2013 2                   31330.0                 30560.0   
5195    1977 0                       0.0           

In [155]:
gsall = [*gs014, *gs1529, *gs3044, *gs4559, *gs60plus]

In [213]:
group = deaths.groupby(level=2)

res = {}

for g, _ in group:
    df = group.get_group(g)
    df.index = df.index.droplevel(2)
    r = rate(df, pop_, pop_div_t,  gsall)
    res[g] = r

In [214]:
to_concat = []

for g, v in res.items():
    v = v.reset_index()
    v['cause'] = g
    v = v.set_index(['country', 'year', 'cause', 'sex'])
    to_concat.append(v)

In [215]:
rate_all = pd.concat(to_concat)

In [216]:
rate_all.dropna().reset_index().query('country == "1125"').year.unique()

array([1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 2004, 2005,
       2006, 2007, 2008, 2011, 2014, 2015])

In [217]:
rate_all.columns = ['deaths_rate_all_ages']

In [218]:
from ddf_utils.str import format_float_digits

In [219]:
(rate_all.replace([np.inf, -np.inf], np.nan)
 .dropna()
 .applymap(format_float_digits)
 .sort_index()
 .to_csv('../../ddf--datapoints--deaths_rate_all_ages--by--country--year--cause--sex.csv'))

In [212]:
!open ../../

In [233]:
# alternative rate_all

In [249]:
d = deaths['deaths_all_ages']
p = pop_['population_all_ages']

In [252]:
def _rate(df):
    df_ = df.copy()
    df_.index = df_.index.droplevel(2)
    return  df_ / p * 100000

In [265]:
gs_ = d.groupby(level=2)

res = []

for g_, _ in gs_:
    df = gs_.get_group(g_)
    df_new = _rate(df)
    df_new = df_new.reset_index()
    df_new['cause'] = g_
    df_new = df_new.set_index(['country', 'year', 'cause', 'sex'])
    res.append(df_new)

In [266]:
rate_all_alt = pd.concat(res)

In [267]:
rate_all_alt.columns = ['deaths_rate_all_ages_alt']

In [268]:
(rate_all_alt.replace([np.inf, -np.inf], np.nan)
 .dropna()
 .applymap(format_float_digits)
 .sort_index()
 .to_csv('../../ddf--datapoints--deaths_rate_all_ages_alt--by--country--year--cause--sex.csv'))

In [307]:
# 014

group = deaths.groupby(level=2)

res = {}

for g, _ in group:
    df = group.get_group(g)
    df.index = df.index.droplevel(2)
    r = rate(df, pop_, pop_div_t,  gs014)
    res[g] = r
    
to_concat = []

for g, v in res.items():
    v = v.reset_index()
    v['cause'] = g
    v = v.set_index(['country', 'year', 'cause', 'sex'])
    to_concat.append(v)
    
rate_014 = pd.concat(to_concat)

In [308]:
rate_014.dropna().reset_index().query('country == "1125"').year.unique()

array([1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965,
       1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1987, 1966, 1967,
       1970, 1971, 1972, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       2009, 2010, 2011, 2012, 2013, 2014, 2015])

In [309]:
rate_014.columns = ['deaths_rate_ages_0_14_years']

In [310]:
(rate_014.replace([np.inf, -np.inf], np.nan)
 .dropna()
 .applymap(format_float_digits)
 .sort_index()
 .to_csv('../../ddf--datapoints--deaths_rate_ages_0_14_years--by--country--year--cause--sex.csv'))

In [226]:
# 1529

In [229]:
group = deaths.groupby(level=2)

res = {}

for g, _ in group:
    df = group.get_group(g)
    df.index = df.index.droplevel(2)
    r = rate(df, pop_, pop_div_t,  gs1529)
    res[g] = r
    
to_concat = []

for g, v in res.items():
    v = v.reset_index()
    v['cause'] = g
    v = v.set_index(['country', 'year', 'cause', 'sex'])
    to_concat.append(v)
    
rate_1529 = pd.concat(to_concat)

In [230]:
rate_1529.dropna().reset_index().query('country == "1125"').year.unique()

array([1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965,
       1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1987, 1966, 1967,
       1970, 1971, 1972, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       2009, 2010, 2011, 2012, 2013, 2014, 2015])

In [231]:
rate_1529.columns = ['deaths_rate_ages_15_29_years']

In [232]:
(rate_1529.replace([np.inf, -np.inf], np.nan)
 .dropna()
 .applymap(format_float_digits)
 .sort_index()
 .to_csv('../../ddf--datapoints--deaths_rate_ages_15_29_years--by--country--year--cause--sex.csv'))

In [269]:
for n1, n2 in zip([gs3044, gs4559, gs60plus], ['30_44', '45_59', '60plus']):
    group = deaths.groupby(level=2)

    res = {}

    for g, _ in group:
        df = group.get_group(g)
        df.index = df.index.droplevel(2)
        r = rate(df, pop_, pop_div_t,  n1)
        res[g] = r

    to_concat = []

    for g, v in res.items():
        v = v.reset_index()
        v['cause'] = g
        v = v.set_index(['country', 'year', 'cause', 'sex'])
        to_concat.append(v)

    rate_g = pd.concat(to_concat)
    
    rate_g.columns = [f'deaths_rate_ages_{n2}_years']
    
    (rate_g.replace([np.inf, -np.inf], np.nan)
     .dropna()
     .applymap(format_float_digits)
     .sort_index()
     .to_csv(f'../../ddf--datapoints--deaths_rate_ages_{n2}_years--by--country--year--cause--sex.csv'))

In [297]:
measures = [f'deaths_rate_ages_{x}_years' for x in ['0_14', '15_29', '30_44', '45_59', '60plus']]

In [298]:
measures.append('deaths_rate_all_ages')
measures.append('deaths_rate_all_ages_alt')

In [299]:
names = [f'Age-standardized deaths rate per 100000 people aged {x} years' 
         for x in ['0-14', '15-29', '30-44', '45-59', '60plus']]

In [300]:
names.append('Age-standardized deaths rate for people all ages')
names.append('crude deaths rate for people all ages')

In [301]:
measures_df = pd.DataFrame({'concept': measures})

In [302]:
measures_df['concept_type'] = 'measure'
measures_df['name'] = names

In [303]:
measures_df

,concept,concept_type,name
0,deaths_rate_ages_0_14_years,measure,Age-standardized deaths rate per 100000 people...
1,deaths_rate_ages_15_29_years,measure,Age-standardized deaths rate per 100000 people...
2,deaths_rate_ages_30_44_years,measure,Age-standardized deaths rate per 100000 people...
3,deaths_rate_ages_45_59_years,measure,Age-standardized deaths rate per 100000 people...
4,deaths_rate_ages_60plus_years,measure,Age-standardized deaths rate per 100000 people...
5,deaths_rate_all_ages,measure,Age-standardized deaths rate for people all ages
6,deaths_rate_all_ages_alt,measure,crude deaths rate for people all ages


In [280]:
discrete = [ 'country', 'sex', 'cause', 'year', 'name']
dt = ['entity_domain', 'entity_domain', 'entity_domain', 'time', 'string']

In [281]:
discrete_df = pd.DataFrame({'concept': discrete, 'concept_type': dt, 'name': [x.title() for x in discrete]})

In [282]:
discrete_df

,concept,concept_type,name
0,country,entity_domain,Country
1,sex,entity_domain,Sex
2,cause,entity_domain,Cause
3,year,time,Year
4,name,string,Name


In [304]:
cdf = pd.concat([discrete_df, measures_df], ignore_index=True)

In [306]:
cdf

,concept,concept_type,name
0,country,entity_domain,Country
1,sex,entity_domain,Sex
2,cause,entity_domain,Cause
3,year,time,Year
4,name,string,Name
5,deaths_rate_ages_0_14_years,measure,Age-standardized deaths rate per 100000 people...
6,deaths_rate_ages_15_29_years,measure,Age-standardized deaths rate per 100000 people...
7,deaths_rate_ages_30_44_years,measure,Age-standardized deaths rate per 100000 people...
8,deaths_rate_ages_45_59_years,measure,Age-standardized deaths rate per 100000 people...
9,deaths_rate_ages_60plus_years,measure,Age-standardized deaths rate per 100000 people...


In [305]:
cdf.to_csv('../../ddf--concepts.csv', index=False)

In [286]:
country = pd.read_csv('../source/country_codes.zip')

In [288]:
country.to_csv('../../ddf--entities--country.csv', index=False)

In [291]:
sex = pd.DataFrame({'sex': [0, 1, 2], 'name': ['both_sexes', 'male', 'female']})

In [292]:
sex

,sex,name
0,0,both_sexes
1,1,male
2,2,female


In [293]:
sex = sex.to_csv('../../ddf--entities--sex.csv', index=False)

In [294]:
cause = pd.DataFrame({'cause': ['traffic', 'suicide', 'homicide'], 
                      'name': ['Road traffic accidents', 'Self-inflicted injuries', 'Homicide']})

In [295]:
cause

,cause,name
0,traffic,Road traffic accidents
1,suicide,Self-inflicted injuries
2,homicide,Homicide


In [296]:
cause.to_csv('../../ddf--entities--cause.csv', index=False)